# Base Histórica Ibovespa
- Dados de cotação diários de 28_12_2022 até 28_12_2025
- Dados tratados por Mateus Moraes da Silva

In [0]:
%run "/Workspace/Users/mateusmoraesds@gmail.com/modelo_previsao_cotacao_ibovespa/Ingestao_bibliotecas_padrao"


In [0]:
## Tabela de origem CSV extraído do site Investing.com

tabela = spark.table("previsao_ibovespa.default.dados_historicos_ibovespa")


In [0]:
tabela.createOrReplaceTempView("historico_ibov")
df = spark.sql("""
    SELECT 
        DATA, 
        `Último`                                  AS ULTIMO,
        Abertura                                  AS ABERTURA,
        `Máxima`                                  AS MAXIMA,
        `Mínima`                                  AS MINIMA,
        REPLACE(REPLACE(`Vol.`, 'B',''), 'M','')  AS VOLUME,
        REPLACE(`Var%`, '%','')                   AS VARIACAO
    FROM historico_ibov
""")
df.createOrReplaceTempView("historico_ibov")

In [0]:
df = spark.sql("""
    SELECT 
        DATA, 
        ULTIMO,
        ABERTURA,
        MAXIMA,
        MINIMA,
        ROUND(CAST(REPLACE(VOLUME,',','.') AS DOUBLE),2)       AS VOLUME_BI,
        ROUND(CAST(REPLACE(VARIACAO,',','.') AS DOUBLE),2)     AS VARIACAO
    FROM historico_ibov
""")
df.createOrReplaceTempView("historico_ibov_1")

In [0]:
#Escrita no formato Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("previsao_ibovespa.default.dados_historicos_ibovespa_tratado")